In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

datos = spark.read.csv("data/iris.csv", inferSchema="true", header="true")
datos.show() 

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|        3.0|         1.4|        0.1| setosa|
|         4.3|        3.0|         1.1| 

In [2]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

datos.columns
input_cols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [3]:
assemblesYeison= VectorAssembler(
    inputCols=input_cols,
    outputCol="features"
)
final_data= assemblesYeison.transform(datos)
final_data.show()

+------------+-----------+------------+-----------+-------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|species|         features|
+------------+-----------+------------+-----------+-------+-----------------+
|         5.1|        3.5|         1.4|        0.2| setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4| setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3| setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2| setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2| setosa|[4.4,2.9,1.4,0.2]|
|         4.9|        3.1|         1.5|        0.1| setosa|[4.9,

In [4]:
from pyspark.ml.clustering import KMeans 
from pyspark.ml.evaluation import ClusteringEvaluator

kmeas= KMeans(featuresCol="features", k=4)
model = kmeas.fit(final_data)
model
model.transform(final_data).groupBy("prediction").count().show()
predictions = model.transform(final_data)
predictions.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   50|
|         3|   28|
|         2|   49|
|         0|   23|
+----------+-----+

+------------+-----------+------------+-----------+-------+-----------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|species|         features|prediction|
+------------+-----------+------------+-----------+-------+-----------------+----------+
|         5.1|        3.5|         1.4|        0.2| setosa|[5.1,3.5,1.4,0.2]|         1|
|         4.9|        3.0|         1.4|        0.2| setosa|[4.9,3.0,1.4,0.2]|         1|
|         4.7|        3.2|         1.3|        0.2| setosa|[4.7,3.2,1.3,0.2]|         1|
|         4.6|        3.1|         1.5|        0.2| setosa|[4.6,3.1,1.5,0.2]|         1|
|         5.0|        3.6|         1.4|        0.2| setosa|[5.0,3.6,1.4,0.2]|         1|
|         5.4|        3.9|         1.7|        0.4| setosa|[5.4,3.9,1.7,0.4]|         1|
|         4.6|        3.4|         1.4|       

In [9]:
predictions.groupBy("sepal_length","sepal_width", "petal_length", "petal_width","prediction").count().show()

+------------+-----------+------------+-----------+----------+-----+
|sepal_length|sepal_width|petal_length|petal_width|prediction|count|
+------------+-----------+------------+-----------+----------+-----+
|         4.8|        3.0|         1.4|        0.1|         1|    1|
|         4.4|        3.0|         1.3|        0.2|         1|    1|
|         6.9|        3.1|         5.1|        2.3|         2|    1|
|         6.3|        3.3|         4.7|        1.6|         2|    1|
|         5.8|        2.7|         5.1|        1.9|         2|    2|
|         4.4|        3.2|         1.3|        0.2|         1|    1|
|         6.0|        2.7|         5.1|        1.6|         2|    1|
|         6.5|        3.2|         5.1|        2.0|         2|    1|
|         5.6|        3.0|         4.1|        1.3|         3|    1|
|         5.0|        3.4|         1.6|        0.4|         1|    1|
|         6.5|        3.0|         5.2|        2.0|         2|    1|
|         5.5|        2.6|        